Read file

In [47]:
import requests
import os
import warnings
from tqdm import tqdm
import numpy as np
import zipfile
import rioxarray
import xarray_leaflet
from ipyleaflet import Map, basemaps, LayersControl, WidgetControl
from ipywidgets import FloatSlider
import matplotlib.pyplot as plt
import fsspec
import s3fs
import xarray as xr
ovhBucket = s3fs.S3FileSystem(client_kwargs={'endpoint_url': 'https://s3.SBG.cloud.ovh.net','region_name':"sbg"},
      key='12902bcf746d47b6999045a29c560d3a',secret='8a0a7d67280f4ebdb92e35964bc21228')
store = s3fs.S3Map(root='hdsa/PRS_L2D_STD_20200804111827_20200804111831_0001.zarr/', s3=ovhBucket, check=False)
dataset = xr.open_zarr(store) # open with xarray
rf = dataset['reflectance'] # First we load the reflectance variable
rf = rf.swap_dims({"longitude":"x","latitude":"y"})

In [26]:
rf

,Array,Chunk
Bytes,677.38 MiB,128.00 kiB
Shape,"(234, 1221, 1243)","(1, 256, 256)"
Count,5851 Tasks,5850 Chunks
Type,uint16,numpy.ndarray


Transform to EPSG 4326

In [35]:
from pyproj import Transformer

r3 = rf.isel(wavelength=0,y=slice(0,1000),x=slice(0,1000))

# Retrieve dataset and coordinates

lat = r3.y.data
lon = r3.x.data

# Reproject coordinates
transformer = Transformer.from_crs(32630, 4326) # UTM30 to EPSG 4326
latlon_reprojected = transformer.transform(lon,lat) #Reproject lat lon
new_lon = latlon_reprojected[0]
new_lat = latlon_reprojected[1]

# Assign reprojected coordinates
r3 = r3.assign_coords(y=new_lat)
r3 = r3.assign_coords(x=new_lon)
r3 = r3.assign_coords({"x": new_lon, "y": new_lat})

In [43]:
r3 = rf.isel(wavelength=0,y=slice(0,1000),x=slice(0,1000))

In [44]:
#r3.rio.set_spatial_dims("longitude","latitude",inplace=True)
r3.rio.write_crs("EPSG:32630",inplace=True)
r3.rio.write_nodata(0, inplace=True)

<xarray.DataArray 'reflectance' (y: 1000, x: 1000)>
dask.array<getitem, shape=(1000, 1000), dtype=uint16, chunksize=(256, 256), chunktype=numpy.ndarray>
Coordinates:
    latitude     (y) float64 4.406e+06 4.406e+06 ... 4.436e+06 4.436e+06
    longitude    (x) float64 2.477e+05 2.477e+05 ... 2.776e+05 2.777e+05
    wavelength   float64 402.4
    spatial_ref  int64 0
Dimensions without coordinates: y, x
Attributes:
    addoffset:    0.0
    scalefactor:  1.5259021896696422e-05
    units:        reflectance
    _FillValue:   0

In [45]:
m = Map(center=[40, -6], zoom=4, basemap=basemaps.CartoDB.DarkMatter, interpolation='nearest')
m


Map(center=[40, -6], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_tex…

In [46]:
warnings.filterwarnings("ignore")
r3.leaflet.plot(m, colormap=plt.cm.terrain)


Url()

LocalTileLayer(name='reflectance', options=['attribution', 'detect_retina', 'max_native_zoom', 'max_zoom', 'mi…